In this python notebooks, we train 3 models as per the paper.

1. Bag Of tricks
2. CNN Model
3. CNN 3 layer architecture.

These models are used for 'regular' ICM codes as outputs. 

More details can be found in the paper.

Mount Google Drive

In [1]:
from google.colab import drive
import os
drive.mount("/content/drive/")

mypath = "drive/My Drive/CS598DLHProject"
os.listdir(mypath)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


['data',
 'BOTKeras.ipynb',
 'CNNModel.ipynb',
 'CNN3Layer.ipynb',
 'Reference',
 'storedModels',
 '.ipynb_checkpoints',
 'Preprocess.ipynb',
 'BOTModel.ipynb',
 'RegularYModelReference.ipynb',
 'BOTModelReference.ipynb']

Unzip the data

In [ ]:
# Already unzipped. Not required

# !unzip drive/My\ Drive/CS598DLHProject/data/original_paper_data/data.npz.zip -d drive/My\ Drive/CS598DLHProject/data/original_paper_data/

Archive:  drive/My Drive/CS598DLHProject/data/original_paper_data/data.npz.zip
  inflating: drive/My Drive/CS598DLHProject/data/original_paper_data/data.npz  
  inflating: drive/My Drive/CS598DLHProject/data/original_paper_data/__MACOSX/._data.npz  


Convert the model into relevant attributes. (X, Y_rolled, Y_regular and categories)

In [2]:
# Used to convert input data data.npz into x,cats,rolled,regular but not required now

# import gzip
# import pickle as pkl
# import gc

# import numpy as np

# DATA_DIR = mypath + "/data/"

# DATA_PATH = DATA_DIR + "preprocessed_3004/"
# DATA_NPZ_FILE = f'{DATA_PATH}data.npz'

# print ('reading from saved file Data.npz: ', DATA_NPZ_FILE)
# data = np.load(DATA_NPZ_FILE)
# lst = data.files
# for item in lst:
#     print(item)
#     # print(data[item])

# print("done")
# x = data['x']
# cats = data['cats']
# y_rolled = data['rol_y']
# y_full = data['reg_y']

# print(x.shape)
# print(cats.shape)
# print(y_rolled.shape)
# print(y_full.shape)

reading from saved file Data.npz:  drive/My Drive/CS598DLHProject/data/preprocessed_3004/data.npz
x
cats
reg_y
rol_y
done
(399623, 2199)
(399623, 15)
(399623, 781)
(399623, 4103)


Distribute the data into Train/Test/Validation data and save the data

In [3]:
# Temporary block to convert data.npz into train/val/test

# texts = x
# texts_categories = cats
# regular_labels = y_full
# rolled_labels = y_rolled

# # Split
# s1 = int(.64 * len(texts))
# s2 = int(.8 * len(texts))
# x_train, x_val, x_test = np.split(texts, [s1, s2])
# cats_train, cats_val, cats_test = np.split(texts_categories, [s1, s2])
# reg_y_train, reg_y_val, reg_y_test = np.split(regular_labels, [s1, s2])
# rol_y_train, rol_y_val, rol_y_test = np.split(rolled_labels, [s1, s2])

# np.savez(f'{DATA_PATH}dataTrainValTest.npz',
#           x_train=x_train, x_val=x_val, x_test=x_test,
#           cats_train=cats_train, cats_val=cats_val, cats_test=cats_test,
#           reg_y_train=reg_y_train, reg_y_val=reg_y_val, reg_y_test=reg_y_test,
#           rol_y_train=rol_y_train, rol_y_val=rol_y_val, rol_y_test=rol_y_test)

Pull in data for training

In [2]:
import gzip
import pickle as pkl
import gc

import numpy as np

DATA_DIR = mypath + "/data/"

DATA_PATH = DATA_DIR + "preprocessed_3004/"
DATA_NPZ_FILE = f'{DATA_PATH}dataTrainValTest.npz'

print ('reading from saved file dataTrainValTest.npz: ', DATA_NPZ_FILE)
data = np.load(DATA_NPZ_FILE)
lst = data.files
for item in lst:
    print(item)
    # print(data[item])

print("done")
x_train_load = data['x_train']
x_val_load = data['x_val']
# x_test = data['x_test']

ratio = 1
x_train = x_train_load[:int(ratio*x_train_load.shape[0])]
x_val = x_val_load[:int(ratio*x_val_load.shape[0])]
# x_test = x_test

del x_train_load
del x_val_load

print('x_train: ' , x_train.shape)
print('x_val: ', x_val.shape)


rol_y_train = data['rol_y_train']
rol_y_val = data['rol_y_val']
# rol_y_test = data['rol_y_test']

print('rol_y_train: ', rol_y_train.shape)
print('rol_y_val: ', rol_y_val.shape)
# print('rol_y_test: ', rol_y_test.shape)

y_train = rol_y_train[:int(ratio*rol_y_train.shape[0])]
y_val = rol_y_val[:int(ratio*rol_y_val.shape[0])]

print('y_train: ', y_train.shape)
print('y_val: ', y_val.shape)
# print('reg_y_test: ', reg_y_test.shape)

del data
del lst
del rol_y_train
del rol_y_val


gc.collect()


reading from saved file dataTrainValTest.npz:  drive/My Drive/CS598DLHProject/data/preprocessed_3004/dataTrainValTest.npz
x_train
x_val
x_test
cats_train
cats_val
cats_test
reg_y_train
reg_y_val
reg_y_test
rol_y_train
rol_y_val
rol_y_test
done
x_train:  (255758, 2199)
x_val:  (63940, 2199)
rol_y_train:  (255758, 781)
rol_y_val:  (63940, 781)
y_train:  (255758, 781)
y_val:  (63940, 781)


43

In [3]:

with open(DATA_PATH + 'row_index_dictionary.p', 'rb') as f:
    word_index = pkl.load(f)
    print("Got word index")

Got word index


In [4]:
embedding_matrix = np.load(DATA_PATH + 'embedding_matrix.p', allow_pickle=True)

In [5]:
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, GlobalAveragePooling1D, Conv1D
from tensorflow.keras.callbacks import EarlyStopping

In [8]:
import keras

model = Sequential()
model.add(Embedding(len(word_index) , embedding_matrix.shape[1],
                    weights=[embedding_matrix], input_length=x_train.shape[1],
                    trainable=True))
model.add(GlobalAveragePooling1D())
model.add(Dense(y_train.shape[1], activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[keras.metrics.Precision(), keras.metrics.Recall()])

early_stopping = EarlyStopping(min_delta=.0001, patience=2)
model.fit(x_train, y_train,
          batch_size=32, epochs=25,
          validation_data=[x_val, y_val],
          callbacks=[early_stopping])
# model.save('{}.h5'.format(filename))

Epoch 1/25
7993/7993 [==============================] - 126s 16ms/step - loss: 0.0611 - precision_1: 0.6837 - recall_1: 0.1160 - val_loss: 0.0595 - val_precision_1: 0.8133 - val_recall_1: 0.1141
Epoch 2/25
7993/7993 [==============================] - 124s 16ms/step - loss: 0.0587 - precision_1: 0.7635 - recall_1: 0.1410 - val_loss: 0.0578 - val_precision_1: 0.7729 - val_recall_1: 0.1521
Epoch 3/25
7993/7993 [==============================] - 124s 16ms/step - loss: 0.0571 - precision_1: 0.7627 - recall_1: 0.1678 - val_loss: 0.0565 - val_precision_1: 0.7539 - val_recall_1: 0.1813
Epoch 4/25
7993/7993 [==============================] - 124s 16ms/step - loss: 0.0560 - precision_1: 0.7627 - recall_1: 0.1851 - val_loss: 0.0555 - val_precision_1: 0.7276 - val_recall_1: 0.2108
Epoch 5/25
7993/7993 [==============================] - 124s 16ms/step - loss: 0.0550 - precision_1: 0.7660 - recall_1: 0.1976 - val_loss: 0.0546 - val_precision_1: 0.7644 - val_recall_1: 0.2035
Epoch 6/25
7993/7993 [===

In [9]:
# model.save(DATA_PATH + 'BOTModel.h5')

In [6]:
import keras

model = Sequential()
model.add(Embedding(len(word_index) , embedding_matrix.shape[1],
                    weights=[embedding_matrix], input_length=x_train.shape[1],
                    trainable=True))
model.add(Conv1D(250, 3, activation='relu'))
model.add(GlobalAveragePooling1D())
model.add(Dense(y_train.shape[1], activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[keras.metrics.Precision(), keras.metrics.Recall()])


early_stopping = EarlyStopping(min_delta=.0001, patience=2)
model.fit(x_train, y_train,
          batch_size=32, epochs=25,
          validation_data=[x_val, y_val],
          callbacks=[early_stopping])

Epoch 1/25
7993/7993 [==============================] - 498s 61ms/step - loss: 0.0587 - precision: 0.7078 - recall: 0.1638 - val_loss: 0.0559 - val_precision: 0.7623 - val_recall: 0.1931
Epoch 2/25
7993/7993 [==============================] - 482s 60ms/step - loss: 0.0545 - precision: 0.7595 - recall: 0.2150 - val_loss: 0.0533 - val_precision: 0.7704 - val_recall: 0.2245
Epoch 3/25
7993/7993 [==============================] - 480s 60ms/step - loss: 0.0522 - precision: 0.7684 - recall: 0.2453 - val_loss: 0.0516 - val_precision: 0.7767 - val_recall: 0.2513
Epoch 4/25
7993/7993 [==============================] - 478s 60ms/step - loss: 0.0505 - precision: 0.7780 - recall: 0.2698 - val_loss: 0.0502 - val_precision: 0.7920 - val_recall: 0.2671
Epoch 5/25
7993/7993 [==============================] - 478s 60ms/step - loss: 0.0492 - precision: 0.7868 - recall: 0.2906 - val_loss: 0.0491 - val_precision: 0.7783 - val_recall: 0.2991
Epoch 6/25
7993/7993 [==============================] - 477s 60ms

In [7]:
# model.save(DATA_PATH + 'CNNBaseline.h5')

In [6]:
import keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Conv1D, GlobalMaxPooling1D, concatenate

embedding_input = Input(shape=(x_train.shape[1],), dtype=np.int32)
embedding_layer = Embedding(len(word_index), embedding_matrix.shape[1],
                                weights=[embedding_matrix], input_length=x_train.shape[1],
                                trainable=True)(embedding_input)
x1 = Conv1D(250, 2, activation='relu')(embedding_layer)
x1 = GlobalMaxPooling1D()(x1)
x2 = Conv1D(250, 3, activation='relu')(embedding_layer)
x2 = GlobalMaxPooling1D()(x2)
x3 = Conv1D(250, 4, activation='relu')(embedding_layer)
x3 = GlobalMaxPooling1D()(x3)
x = concatenate([x1, x2, x3])
output = Dense(y_train.shape[1], activation='sigmoid')(x)

model = Model(inputs=[embedding_input], outputs=[output])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[keras.metrics.Precision(), keras.metrics.Recall()])


early_stopping = EarlyStopping(min_delta=.0001, patience=10)
model.fit(x_train, y_train,
          batch_size=32, epochs=25,
          validation_data=[x_val, y_val],
          callbacks=[early_stopping])

Epoch 1/25
7993/7993 [==============================] - 661s 81ms/step - loss: 0.0557 - precision: 0.6923 - recall: 0.2873 - val_loss: 0.0470 - val_precision: 0.7809 - val_recall: 0.3455
Epoch 2/25
7993/7993 [==============================] - 648s 81ms/step - loss: 0.0450 - precision: 0.7934 - recall: 0.3738 - val_loss: 0.0442 - val_precision: 0.7950 - val_recall: 0.3894
Epoch 3/25
7993/7993 [==============================] - 647s 81ms/step - loss: 0.0427 - precision: 0.8119 - recall: 0.4027 - val_loss: 0.0432 - val_precision: 0.8125 - val_recall: 0.3986
Epoch 4/25
7993/7993 [==============================] - 647s 81ms/step - loss: 0.0414 - precision: 0.8212 - recall: 0.4180 - val_loss: 0.0428 - val_precision: 0.8245 - val_recall: 0.4019
Epoch 5/25
7993/7993 [==============================] - 647s 81ms/step - loss: 0.0407 - precision: 0.8266 - recall: 0.4275 - val_loss: 0.0427 - val_precision: 0.8122 - val_recall: 0.4189
Epoch 6/25
7993/7993 [==============================] - 647s 81ms

In [7]:
# model.save(DATA_PATH + 'CNN3LayerV2.h5')